In [6]:
import deepcut
from pathlib import Path
from pythainlp import word_tokenize
from matplotlib import pyplot as plt

import re
import pandas as pd
import numpy as np
from numpy import array
from numpy import argmax

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

from nltk import FreqDist, precision, recall, f_measure, NaiveBayesClassifier
from nltk.classify import apply_features
from nltk.classify import util
import collections, itertools
from sklearn.model_selection import KFold

In [7]:
df = pd.read_csv('train.csv')
df = df.rename(columns={"words":"texts","labels":"category"})
df

,texts,category
0,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ,neu
1,คะ,neu
2,อิเหี้ยออมทำกูอยากกินเอ็มเค,neg
3,😅😅😅,neu
4,สวัสดีวันพุธ แนน อะไรนะ,neu
...,...,...
25610,อีชาติชั้ว,neg
25611,อีช้างเย๊ด,neg
25612,อยากเอาหญิง,neg
25613,ทมิฬ,neg


In [8]:
'''
texts = []
labels = []
with open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train.txt",encoding="utf8") as f:
    for line in f:
        texts.append(line.strip())
        
with open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train_label.txt",encoding="utf8") as f:
    for line in f:
        labels.append(line.strip())

df = pd.DataFrame({"category":labels, "texts":texts})
df.shape
'''

'\ntexts = []\nlabels = []\nwith open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train.txt",encoding="utf8") as f:\n    for line in f:\n        texts.append(line.strip())\n        \nwith open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train_label.txt",encoding="utf8") as f:\n    for line in f:\n        labels.append(line.strip())\n\ndf = pd.DataFrame({"category":labels, "texts":texts})\ndf.shape\n'

In [9]:
df.sample(20)

,texts,category
4580,แหม่ ระดับผบทบ.มานั่งประธาน กระดิกนิ้วนิดเดียว...,neu
14013,ส่งให้ดูยังไม่สนใจ,neu
1120,มื้อเที่ยงวันนี้สาขาบิ๊กซีหาดใหญ่ค่ะ,neu
9943,ราคา 280 💸บาท มาส์กสำหรับผิวรอบดวงตาชนิดแรกของ...,neu
1932,น่าใช้นะ ถ้าไม่ใช่Camry ไม่เคยรู้สึกว่าToyotaน...,pos
20333,ชุมพรมั้ยอ้ะ?!😂,q
22865,นานไปไหม ปี 2020 All New Mazda BT-50 Pro รุ่นใ...,neu
4388,เตรียมพบกับ ใหม่! อีซูซุดีแมคซ์เอ็กซ์-ซีรี่ส์ ...,neu
7442,บ้านแสนสิริ บางหมู่บ้านอย่างแย่ สวยแต่รูป อยู่...,neg
13178,ใหม่! Precious Mineral Compact SPF30/PA++ แป้ง...,neu


In [10]:
df = df[df.category != 'q']

In [11]:
df.groupby('category').describe()

texts                                                               
          count unique                                                top freq
category                                                                      
neg        7393   7393  ไปทานที่เมกะบางนา มา 2 ครั้ง เจอประสบการณ์ไม่ด...    1
neu       13087  13087  ขอวิธี ทาง หนี ที่ไล่ ด้วยครับ ถ้าเจอสถานการณ์...    1
pos        4617   4617                                                แถม    1

In [12]:
df['length'] = df['texts'].apply(word_tokenize).apply(len)
df['length'].describe()

C:\Users\60050204\Anaconda3\envs\BigData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


count    25097.000000
mean        25.850779
std         44.237998
min          1.000000
25%          5.000000
50%         11.000000
75%         28.000000
max        758.000000
Name: length, dtype: float64

In [13]:
df

,texts,category,length
0,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ,neu,13
1,คะ,neu,1
2,อิเหี้ยออมทำกูอยากกินเอ็มเค,neg,9
3,😅😅😅,neu,1
4,สวัสดีวันพุธ แนน อะไรนะ,neu,8
...,...,...,...
25610,อีชาติชั้ว,neg,3
25611,อีช้างเย๊ด,neg,3
25612,อยากเอาหญิง,neg,3
25613,ทมิฬ,neg,1


In [14]:
df.sample(100)

,texts,category,length
8046,พรุ่งนี้ มีสอบจ้างเหมานะสิพี่ 😭😭,neu,11
4799,เอ็มเคจะมาส่งพรุ่งนี้เลยมั้ย กูจะได้ต้มรามยอนก...,neg,26
1462,พฤหัส #ไม่จริงงงน้ะะ ไม่ค่อยยุ่งงเท่าไหร่ #กิน...,neu,28
6258,อือหือ,neu,2
3061,หมดแล้วอะไรจะเร็วขนาดนี้,neg,7
...,...,...,...
20815,การที่ใส่ผ้าอนามัยสูตรเย็นแล้วมาเข้าห้องน้ำเอา...,neg,25
4225,ขายเบียร์ไมวันนี้ แอดมิน,neu,6
12179,อ้วนมากค่ะ,neg,3
24599,ค้าง,neg,1


In [15]:
from gensim.models import Word2Vec 

In [16]:
y_train = [row for row in df['category']]
Y_train = array(y_train)
Y_train.shape

(25097,)

In [17]:
label_encoder = LabelEncoder()

In [18]:
integer_encoder = label_encoder.fit_transform(Y_train)

In [19]:
integer_encoder

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [20]:
ohe = OneHotEncoder(sparse=False)

In [21]:
onehot_encoded = ohe.fit_transform(integer_encoder.reshape(-1,1))

In [22]:
onehot_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [23]:
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(df.texts.values)


In [24]:
word_index = tokenizer.word_index


In [25]:
vocab_size = len(word_index)

In [26]:
vocab_size

87090

In [27]:
df['length'].describe()

count    25097.000000
mean        25.850779
std         44.237998
min          1.000000
25%          5.000000
50%         11.000000
75%         28.000000
max        758.000000
Name: length, dtype: float64

In [28]:
max_length = 758

In [29]:
x_train = tokenizer.texts_to_sequences(df.texts.values)

In [30]:
x_train_padded = pad_sequences(x_train, maxlen=max_length ,padding='post')

In [31]:
x_train_padded.shape

(25097, 758)

In [32]:
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout, Activation
from tensorflow.keras.models import Sequential

In [33]:
model = Sequential()
model.add(Embedding(100000+1,32,input_length=max_length))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 758, 32)           3200032   
_________________________________________________________________
flatten (Flatten)            (None, 24256)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3104896   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               1

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
x_train, x_test, y_train, y_test = train_test_split(x_train_padded, onehot_encoded, test_size=0.2, shuffle=True, random_state=101)
#10fold
#multi model

In [36]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((20077, 758), (20077, 3), (5020, 758), (5020, 3))

In [37]:
x_train.shape

(20077, 758)

In [38]:
model.fit(x_train,y_train,
         batch_size=20,
         epochs=3,)

Train on 20077 samples
Epoch 1/3
20077/20077 [==============================] - 50s 3ms/sample - loss: 0.9834 - accuracy: 0.5233
Epoch 2/3
20077/20077 [==============================] - 45s 2ms/sample - loss: 0.6620 - accuracy: 0.7198
Epoch 3/3
20077/20077 [==============================] - 64s 3ms/sample - loss: 0.3344 - accuracy: 0.8625


In [39]:
test_loss,test_acc = model.evaluate(x_test,y_test)
print("\nTest accuracy:",test_acc)
print("\nTest Loss:",test_loss)


5020/5020 [==============================] - 1s 251us/sample - loss: 1.5206 - accuracy: 0.3418

Test accuracy: 0.34183267

Test Loss: 1.5205848508622066


In [40]:
predicions = model.predict(x_test)
predicions

array([[0.08672076, 0.32930368, 0.5839756 ],
       [0.0118133 , 0.8987223 , 0.08946445],
       [0.6574621 , 0.01448253, 0.32805526],
       ...,
       [0.18298605, 0.11368454, 0.7033294 ],
       [0.13286783, 0.15713805, 0.70999414],
       [0.1412177 , 0.14661147, 0.7121709 ]], dtype=float32)

In [41]:
reverse_ohe = ohe.inverse_transform(predicions)
reverse_label = label_encoder.inverse_transform(reverse_ohe)
reverse_label
outputDataFrame = pd.DataFrame(reverse_label,columns={'label'})

C:\Users\60050204\Anaconda3\envs\BigData\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [42]:
reverse_x_test = tokenizer.sequences_to_texts(x_test)
x_test_DF = pd.DataFrame(reverse_x_test,columns={'words'})

sum_df = x_test_DF.join(outputDataFrame)
sum_df


,words,label
0,เดอะเฟสช็อปนี่ใช้คุชชั่น เสียดายที่กัซไม่ได้โฆ...,pos
1,รบกวนสอบถาม เรื่องการต่ออายุบัตรสมาชิก ทาง mk ...,neu
2,ไม่เคยกล้าซื้อครีมที่ขายตามเน๊ตอ่ะ คือแบบ กูกล...,neg
3,ของการนอน,pos
4,ถึง29,pos
...,...,...
5015,ดีงามมาก ต้อง รามอินทรา,neu
5016,ผมถามหน่อยทุกท่านที่เล่น พร้อมสูบจริงๆ น้ำยาด้...,pos
5017,ตอแหลป่าว,pos
5018,พุกนี้ ศุกหรรษา จัดเรยก่,pos


In [43]:
test_word = 'กล้าพูดเนอะ'
test_word_df = pd.DataFrame([test_word],columns=['texts'])
test_word_df

,texts
0,กล้าพูดเนอะ


In [44]:
test_word_encoded = tokenizer.texts_to_sequences(test_word_df.texts.values)
test_word_padded = pad_sequences(test_word_encoded, maxlen=max_length ,padding='post')
test_word_encoded

[[67864]]

In [45]:
pred = model.predict(test_word_padded)
pred

array([[0.31856638, 0.05295878, 0.62847483]], dtype=float32)

In [46]:
reverse_ohe = ohe.inverse_transform(pred)
reverse_label = label_encoder.inverse_transform(reverse_ohe)
reverse_label

C:\Users\60050204\Anaconda3\envs\BigData\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


array(['pos'], dtype='<U3')